##### Extract text (from an article) from a popular newspaper for the related topic. Create 5 separate documents. Find the similarity measure (cosine similarity) between each one of them after weighting the terms as TF-IDF 

In [ ]:
import pandas as pd
import numpy as np
import requests

In [ ]:
link = "https://www.forbes.com/sites/alexledsom/2020/11/10/post-covid-19-travel-41-people-worldwide-expect-to-travel-less-by-any-meansnew-research/?sh=688647c65f67"

link for news article for data extraction.

In [ ]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head',  'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

In [ ]:
from bs4 import BeautifulSoup
from bs4.element import Comment
req  = requests.get(link, timeout=5)
data = req.text
soup = BeautifulSoup(data, "html.parser")
texts = soup.findAll(text=True)
visible_texts = filter(tag_visible, texts)  
text = u" ".join(t.strip() for t in visible_texts)

extracting text from the wesite and inserting it in text

In [ ]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
wt=tokenizer.tokenize(text)

creating 5 different doccuments with this content

In [ ]:
len(wt)//5

298

In [ ]:
doc=[]
s=0
e=297
for i in range(5):
  doc.append(wt[s:e])
  s+=298
  e+=298

In [ ]:
total_unique_words = set(wt[:-4])

creating dictionaries with total unique words

In [ ]:
doc1 = dict.fromkeys(total_unique_words, 0) 
doc2 = dict.fromkeys(total_unique_words, 0)
doc3 = dict.fromkeys(total_unique_words, 0)
doc4 = dict.fromkeys(total_unique_words, 0)
doc5 = dict.fromkeys(total_unique_words, 0)

calculating count of the words

In [ ]:
for i in total_unique_words:
  doc1[i] = doc[0].count(i)
  doc2[i] = doc[1].count(i)
  doc3[i] = doc[2].count(i)
  doc4[i] = doc[3].count(i)
  doc5[i] = doc[4].count(i)

creating data frame for all 5 documnets

In [ ]:
pd.DataFrame([doc1, doc2, doc3, doc4, doc5])

,fairly,news,three,Depot,greater,LinkedIn,32,Women,Tucson,absorb,Analytics,Back,Thailand,President,This,respectively,passengers,APAC,Explore,37am,Plus,conducted,Billionaires,Italy,virus,29,Hackathon,Watches,Coverage,worldwide,Unsurprisingly,Korea,Social,mobile,Careers,age,Energy,Jill,Aka,much,...,Wheels,SportsMoney,just,generally,highly,before,Hollywood,Remote,Hands,Impact,Rights,clearly,our,EY,forth,stages,Sign,Less,World,global,has,Where,recent,To,Taiwan,be,Honeywell,AdChoices,Healthcare,Cards,delivers,Crowe,re,S,Drink,even,year,BrandVoice,5G,Featured
0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,3,0,0,0,0,1,0,0,0,1,1,0,1,0,1,0,0,0,...,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,1,0,1,0,0,1,0,0,1,0,0,17,1,0
1,0,0,0,1,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,...,2,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,8,0,0,0,0,0,0,0,4,0,1
2,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,4,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,...,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,2,3,1,1,1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,1,0,1,0,1,0,0,0,0,2,1,0,1,0,0,0,1,1,0,...,0,0,1,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0
4,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


creating tf function

In [ ]:
def computeTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word] = count/float(bowCount)
    return tfDict

tf1 = computeTF(doc1, doc[0])
tf2 = computeTF(doc2, doc[1])
tf3 = computeTF(doc3, doc[2])
tf4 = computeTF(doc4, doc[3])
tf5 = computeTF(doc5, doc[4])

#Converting to dataframe for visualization
tf_df= pd.DataFrame([tf1, tf2, tf3, tf4,tf5])
tf_df

,fairly,news,three,Depot,greater,LinkedIn,32,Women,Tucson,absorb,Analytics,Back,Thailand,President,This,respectively,passengers,APAC,Explore,37am,Plus,conducted,Billionaires,Italy,virus,29,Hackathon,Watches,Coverage,worldwide,Unsurprisingly,Korea,Social,mobile,Careers,age,Energy,Jill,Aka,much,...,Wheels,SportsMoney,just,generally,highly,before,Hollywood,Remote,Hands,Impact,Rights,clearly,our,EY,forth,stages,Sign,Less,World,global,has,Where,recent,To,Taiwan,be,Honeywell,AdChoices,Healthcare,Cards,delivers,Crowe,re,S,Drink,even,year,BrandVoice,5G,Featured
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003367,0.000000,0.000000,0.000000,0.000000,0.003367,0.000000,0.000000,0.000000,0.000000,0.000000,0.003367,0.0,0.000000,0.000000,0.010101,0.000000,0.000000,0.000000,0.000000,0.003367,0.000000,0.000000,0.000000,0.003367,0.003367,0.000000,0.003367,0.000000,0.003367,0.000000,0.000000,0.000000,...,0.000000,0.003367,0.000000,0.000000,0.000000,0.000000,0.003367,0.003367,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003367,0.003367,0.000000,0.000000,0.000000,0.000000,0.003367,0.003367,0.000000,0.003367,0.000000,0.003367,0.000000,0.000000,0.003367,0.000000,0.000000,0.003367,0.000000,0.000000,0.057239,0.003367,0.000000
1,0.000000,0.000000,0.000000,0.003367,0.000000,0.000000,0.000000,0.000000,0.003367,0.000000,0.003367,0.003367,0.000000,0.000000,0.003367,0.000000,0.000000,0.000000,0.000000,0.0,0.003367,0.000000,0.000000,0.000000,0.000000,0.000000,0.003367,0.000000,0.003367,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003367,0.000000,0.000000,0.000000,...,0.006734,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003367,0.003367,0.003367,0.000000,0.000000,0.003367,0.000000,0.000000,0.003367,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003367,0.000000,0.026936,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013468,0.000000,0.003367
2,0.003367,0.000000,0.000000,0.000000,0.003367,0.000000,0.003367,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013468,0.000000,0.000000,0.0,0.000000,0.003367,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003367,0.000000,0.000000,0.000000,0.000000,0.000000,0.003367,0.000000,0.000000,0.000000,0.003367,...,0.000000,0.000000,0.000000,0.003367,0.003367,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003367,0.000000,0.000000,0.000000,0.000000,0.000000,0.003367,0.000000,0.006734,0.010101,0.003367,0.003367,0.003367,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003367,0.000000,0.000000,0.003367,0.003367,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.003367,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003367,0.000000,0.003367,0.003367,0.003367,0.000000,0.0,0.000000,0.003367,0.000000,0.003367,0.000000,0.003367,0.000000,0.000000,0.000000,0.000000,0.006734,0.003367,0.000000,0.003367,0.000000,0.000000,0.000000,0.003367,0.003367,0.000000,...,0.000000,0.000000,0.003367,0.000000,0.003367,0.003367,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003367,0.000000,0.000000,0.000000,0.000000,0.003367,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003367,0.000000,0.000000,0.003367,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.003367,0.000000,0.000000,0.000000,0.003367,0.000000,0.000000,0.000000,0.003367,0.000000,0.000000,0.000000,0.003367,0.000000,0.000000,0.010101,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.003367,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.003367,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003367,0.000000,0.000000,0.003367,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

In [ ]:
import math
def computeIDF(docList):
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        if val > 0:
          idfDict[word] = math.log10(N / float(val))
        
    return idfDict
#inputing our sentences in the log file
idfs = computeIDF([doc1, doc2,doc3,doc4,doc5])

#The actual calculation of TF*IDF from the table above:
def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf
#running our two sentences through the IDF:
idf1 = computeTFIDF(tf1, idfs)
idf2 = computeTFIDF(tf2, idfs)
idf3 = computeTFIDF(tf3, idfs)
idf4 = computeTFIDF(tf4, idfs)
idf5 = computeTFIDF(tf5, idfs)

idf= pd.DataFrame([idf1,idf2,idf3,idf4,idf5])
idf

,fairly,news,three,Depot,greater,LinkedIn,32,Women,Tucson,absorb,Analytics,Back,Thailand,President,This,respectively,passengers,APAC,Explore,37am,Plus,conducted,Billionaires,Italy,virus,29,Hackathon,Watches,Coverage,worldwide,Unsurprisingly,Korea,Social,mobile,Careers,age,Energy,Jill,Aka,much,...,Wheels,SportsMoney,just,generally,highly,before,Hollywood,Remote,Hands,Impact,Rights,clearly,our,EY,forth,stages,Sign,Less,World,global,has,Where,recent,To,Taiwan,be,Honeywell,AdChoices,Healthcare,Cards,delivers,Crowe,re,S,Drink,even,year,BrandVoice,5G,Featured
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002353,0.000000,0.000000,0.000000,0.000000,0.002353,0.00000,0.000000,0.000000,0.000000,0.000000,0.002353,0.0,0.000000,0.00000,0.00706,0.000000,0.000000,0.000000,0.000000,0.002353,0.000000,0.000000,0.000000,0.00134,0.002353,0.000000,0.002353,0.000000,0.00134,0.000000,0.000000,0.000000,...,0.000000,0.002353,0.000000,0.000000,0.00000,0.00000,0.002353,0.002353,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00134,0.002353,0.00000,0.00000,0.000000,0.000000,0.00134,0.002353,0.000000,0.002353,0.000000,0.002353,0.000000,0.000000,0.002353,0.000000,0.000000,0.002353,0.000000,0.000000,0.022778,0.002353,0.000000
1,0.000000,0.000000,0.000000,0.002353,0.000000,0.000000,0.000000,0.000000,0.002353,0.000000,0.002353,0.002353,0.000000,0.00000,0.002353,0.000000,0.000000,0.000000,0.000000,0.0,0.002353,0.00000,0.00000,0.000000,0.000000,0.000000,0.002353,0.000000,0.002353,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00134,0.000000,0.000000,0.000000,...,0.004707,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.002353,0.002353,0.002353,0.000000,0.000000,0.002353,0.000000,0.000000,0.002353,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.002353,0.000000,0.018827,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005359,0.000000,0.002353
2,0.002353,0.000000,0.000000,0.000000,0.002353,0.000000,0.002353,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.002988,0.000000,0.000000,0.0,0.000000,0.00134,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002353,0.000000,0.00000,0.000000,0.000000,0.000000,0.002353,0.00000,0.000000,0.000000,0.002353,...,0.000000,0.000000,0.000000,0.002353,0.00134,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002353,0.000000,0.000000,0.000000,0.000000,0.000000,0.00134,0.000000,0.00268,0.00706,0.002353,0.002353,0.00134,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002353,0.000000,0.000000,0.002353,0.002353,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.002353,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00134,0.000000,0.002353,0.000747,0.002353,0.000000,0.0,0.000000,0.00134,0.00000,0.002353,0.000000,0.002353,0.000000,0.000000,0.000000,0.000000,0.004707,0.00134,0.000000,0.002353,0.000000,0.000000,0.00000,0.002353,0.002353,0.000000,...,0.000000,0.000000,0.002353,0.000000,0.00134,0.00134,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002353,0.000000,0.000000,0.00000,0.000000,0.00134,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002353,0.000000,0.000000,0.002353,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.002353,0.000000,0.000000,0.000000,0.002353,0.000000,0.000000,0.000000,0.002353,0.000000,0.000000,0.000000,0.00134,0.000000,0.000000,0.002241,0.000000,0.000000,0.0,0.000000,0.00000,0.00000,0.000000,0.002353,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00134,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002353,0.000000,0.000000,0.002353,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.002353,0.000000,0.00000

In [ ]:
from numpy import dot
from numpy.linalg import norm

In [ ]:
keys= ["doc1","doc2","doc3","doc4","doc5"]
d1=dict.fromkeys(keys, 0)
d2=dict.fromkeys(keys, 0)
d3=dict.fromkeys(keys, 0)
d4=dict.fromkeys(keys, 0)
d5=dict.fromkeys(keys, 0)

cosine similarity

In [ ]:
k=0
a1=idf.iloc[0].to_numpy()
a2=idf.iloc[1].to_numpy()
a3=idf.iloc[2].to_numpy()
a4=idf.iloc[3].to_numpy()
a5=idf.iloc[4].to_numpy()
for i in keys:
  b = idf.iloc[k].to_numpy()
  d1[i] = dot(a1, b)/(norm(a1)*norm(b))
  d2[i] = dot(a2, b)/(norm(a2)*norm(b))
  d3[i] = dot(a3, b)/(norm(a3)*norm(b))
  d4[i] = dot(a4, b)/(norm(a4)*norm(b))
  d5[i] = dot(a5, b)/(norm(a5)*norm(b))
  k+=1

In [ ]:
pd.DataFrame([d1,d2,d3,d4,d5],index=keys)

,doc1,doc2,doc3,doc4,doc5
doc1,1.000000,0.281093,0.019365,0.005397,0.002566
doc2,0.281093,1.000000,0.011021,0.006680,0.003613
doc3,0.019365,0.011021,1.000000,0.244401,0.210861
doc4,0.005397,0.006680,0.244401,1.000000,0.215057
doc5,0.002566,0.003613,0.210861,0.215057,1.000000
